In [2]:
# !pip install opensubtitlescom

In [3]:
# method to download and save imdb data to data/raw/
import requests

def download_data(url, filename):
    response = requests.get(url)
    with open(filename, 'wb') as file:
        file.write(response.content)

url = 'https://datasets.imdbws.com/'
basics = 'title.basics.tsv.gz'
ratings = 'title.ratings.tsv.gz'
path = '../data/raw/'

# download_data(f'{url}{ratings}', f'{path}{ratings}')
# download_data(f'{url}{basics}', f'{path}{basics}')

In [4]:
import pandas as pd

# load the data
basics_df = pd.read_csv(f'{path}{basics}', sep='\t', na_values='\\N')
ratings_df = pd.read_csv(f'{path}{ratings}', sep='\t', na_values='\\N')

C:\Users\kyle0\AppData\Local\Temp\ipykernel_2848\1510432930.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  basics_df = pd.read_csv(f'{path}{basics}', sep='\t', na_values='\\N')


In [5]:
# filter out only movies
movies_df = basics_df[basics_df['titleType'] == 'movie']

In [6]:
movies_df = movies_df.merge(ratings_df, on='tconst')

In [7]:
# sort by number of votes
movies_df = movies_df.sort_values('numVotes', ascending=False)

In [11]:
# remove adult movies
movies_df = movies_df[movies_df['isAdult'] == 0]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,imdb_id
61716,tt0111161,movie,The Shawshank Redemption,The Shawshank Redemption,0.0,1994.0,NaN,142.0,Drama,9.3,2880568,111161
137188,tt0468569,movie,The Dark Knight,The Dark Knight,0.0,2008.0,NaN,152.0,"Action,Crime,Drama",9.0,2863134,468569
178099,tt1375666,movie,Inception,Inception,0.0,2010.0,NaN,148.0,"Action,Adventure,Sci-Fi",8.8,2543775,1375666
71673,tt0137523,movie,Fight Club,Fight Club,0.0,1999.0,NaN,139.0,Drama,8.8,2316350,137523
60983,tt0109830,movie,Forrest Gump,Forrest Gump,0.0,1994.0,NaN,142.0,"Drama,Romance",8.8,2250065,109830


In [9]:
# make new column called imdb_id. This is the same as tconst but with 'tt' removed
movies_df['imdb_id'] = movies_df['tconst'].str[2:]

# now remove leading zeroes (for the subtitle api)
movies_df['imdb_id'] = movies_df['imdb_id'].str.lstrip('0')

In [12]:
df = movies_df[['tconst', 'imdb_id', 'primaryTitle', 'genres', 'startYear', 'averageRating', 'numVotes']]
df = df.reset_index(drop=True)

# drop startYear if it is NaN
df = df.dropna(subset=['startYear'])

df['startYear'] = df['startYear'].astype(int)
df['averageRating'] = df['averageRating'].astype(float)
df['numVotes'] = df['numVotes'].astype(int)

# save as tsv
df[:10000].to_csv('../data/processed/movies.tsv', sep='\t', index=False)

In [1]:
from opensubtitlescom import OpenSubtitles
import pandas as pd
import json
import os
from dotenv import load_dotenv
load_dotenv()  # Loads variables from the .env file

APP_NAME = os.getenv('APP_NAME')
API_KEY = os.getenv('API_KEY')
MY_USERNAME = os.getenv('MY_USERNAME')
MY_PASSWORD = os.getenv('MY_PASSWORD')

# Initialize the OpenSubtitles client
subtitles = OpenSubtitles(APP_NAME, API_KEY)
subtitles.login(MY_USERNAME, MY_PASSWORD)

df = pd.read_csv('../data/processed/movies.tsv', sep='\t')

In [2]:
# list movie ids
ids = df['imdb_id'][:415].tolist()
# ids

In [3]:
subtitle_path = "../data/raw/subtitles/"
subs_list = os.listdir(subtitle_path)

for i in ids:
    prefix = f'{subtitle_path}{i} - '  # Construct the base part of the filename

    sub_found = False

    # Check if subtitles already exist
    for file in os.listdir(subtitle_path):
        if file.startswith(f'{i} - '):
            sub_found = True
            break

    if sub_found:
        # print(f'Subtitles for {i} already exist')
        continue

    # Search for subtitles based on imdb_id
    response = subtitles.search(imdb_id=i, languages="en", order_by="download_count", order_direction="desc")

    # Convert the response to a Json format
    response_json = response.to_json()
    r = json.loads(response_json)
    title = r['data'][0]['title'] # Get the title of the movie

    # Remove special characters but keep spaces
    title = ''.join(e for e in title if e.isalnum() or e.isspace())

    # Download and parse the subtitle object
    srt = subtitles.download_and_parse(response.data[0])

    # Save
    with open(f'{prefix}{title}.txt', 'w', encoding='utf-8') as f:
        for i in srt:
            # remove '<i>' and '</i>' tags
            i.content = i.content.replace('<i>', '').replace('</i>', '')
            f.write(i.content + '\n')
